Article : https://larevueia.fr/langchain-le-guide-essentiel/

# Concept 1 : les prompts dans Langchain

In [ ]:
%pip install langchain langchain-openai langchain-chroma beautifulsoup4 langchain-community langgraph

## Syntaxe LCEL

In [2]:
import os
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

# 1. Configuration de la clé API (Variable d'environnement)
load_dotenv()

# 2. Création du Modèle (ChatModel)
# Nous utilisons ChatOpenAI qui est optimisé pour la conversation
llm = ChatOpenAI(model_name="gpt-5")

# 3. Création du Template
template = """
Question de l'utilisateur : {question}
Réponse :
"""
prompt = PromptTemplate.from_template(template)

# 4. Création de la Chaîne (Syntaxe LCEL)
# C'est ici que la magie opère : Prompt -> Modèle
chain = prompt | llm

# 5. Exécution
reponse = chain.invoke({"question": "Qui a gagné la coupe du monde 2022 au Qatar ?"})
print(reponse.content)

L’Argentine. Elle a battu la France en finale (3-3, 4-2 t.a.b.) le 18 décembre 2022.


## Prompt avec contexte

In [ ]:
template_contexte = """
Utilisez uniquement le contexte suivant pour répondre à la question.
Si vous ne connaissez pas la réponse d'après le contexte, dites "Je ne sais pas".

Contexte :
LangChain est un framework open-source lancé en 2022 par Harrison Chase.
Il permet de connecter des LLM à des sources de données externes.
En 2025, la fonctionnalité phare est LangGraph pour créer des agents cycliques.

Question : {question}
"""

prompt_rag = PromptTemplate.from_template(template_contexte)

# On réutilise le même modèle 'llm' défini plus haut
rag_chain = prompt_rag | llm

print(rag_chain.invoke({"question": "Qui a lancé LangChain ?"}).content)

Harrison Chase.


## Few-shot prompting

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# 1. Définition de nos exemples
exemples = [
    {"input": "Comment ça va ?", "output": "Je ne peux pas me plaindre, mais je le fais quand même."},
    {"input": "Quelle heure est-il ?", "output": "L'heure de s'acheter une montre."},
]

# 2. Création du format pour les exemples
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# 3. Injection des exemples dans le prompt final
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=exemples,
)

# 4. Assemblage du prompt final
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Tu es un assistant sarcastique et plein d'esprit."),
        few_shot_prompt, # On insère les exemples ici
        ("human", "{input}"),
    ]
)

# 5. Création et exécution de la chaîne
sarcastic_chain = final_prompt | llm

print(sarcastic_chain.invoke({"input": "Quel est le sens de la vie ?"}).content)

42. Mais comme le SAV de l’existence ne décroche jamais, on bricole.

Plus sérieusement: le sens, c’est un mix de
- Lien: aimer des gens sans mode d’emploi.
- Curiosité: apprendre des trucs inutiles… puis les utiliser.
- Contribution: laisser un endroit un peu mieux qu’on l’a trouvé.
- Joie: savourer les petites victoires et le bon café.

Choisis-en deux pour aujourd’hui, recommence demain. Et bois de l’eau.


# Concept 2 : construire un RAG avec Langchain

## Étape 1 : Indexation (préparation des données)

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# 1. Chargement : On récupère le contenu d'un article Web
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_="post-content"))
)
docs = loader.load()

# 2. Découpage : On coupe le texte en morceaux de 1000 caractères
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# 3. Stockage Vectoriel : On transforme le texte en vecteurs et on indexe dans Chroma
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# On crée un "retriever" : l'outil capable de faire la recherche
retriever = vectorstore.as_retriever()

## Étape 2 : La chaîne de génération (LCEL)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Initialisation du modèle (Le dernier cri : GPT-5)
llm = ChatOpenAI(model_name="gpt-5")

# Définition du prompt système spécial RAG
template = """Vous êtes un assistant de recherche expert.
Utilisez le contexte suivant pour répondre à la question.
Si vous ne connaissez pas la réponse, dites simplement que vous ne savez pas.
Soyez concis.

Contexte : {context}

Question : {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Petite fonction utilitaire pour joindre les documents trouvés en un seul texte
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# LA MAGIE DU LCEL (La chaîne RAG complète)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## Étape 3 : L'exécution

In [ ]:
# On pose une question sur le contenu spécifique de l'article
response = rag_chain.invoke("Qu'est-ce que la décomposition des tâches dans les agents autonomes ?")

print(response)

La décomposition des tâches est la pratique qui consiste à fractionner une tâche complexe en sous-tâches plus petites et gérables afin de mieux planifier et raisonner. Dans les agents autonomes, elle se fait notamment via:
- Chain of Thought (penser étape par étape) pour dérouler une suite d’étapes simples.
- Tree of Thoughts, qui explore plusieurs pistes de raisonnement à chaque étape (arbre de pensées, recherche BFS/DFS avec évaluation).
- Des invites dédiées (“Steps for X…”, “Subgoals…”) ou des instructions spécifiques (ex. “écrire un plan”), voire des apports humains.

Objectif: faciliter la planification long-horizon, améliorer la performance et rendre le raisonnement plus interprétable. (À distinguer de LLM+P, qui délègue la planification à un planificateur externe via PDDL.)


# Concept 3 : agents

## Création des outils (les super-pouvoirs de l'agent)

In [ ]:
from langchain_core.tools import tool

@tool
def multiplier(a: float, b: float) -> float:
    """Multiplie deux nombres. Utile pour les calculs précis."""
    return a * b

@tool
def compter_lettres(mot: str) -> int:
    """Compte le nombre de lettres dans un mot."""
    return len(mot)

tools = [multiplier, compter_lettres]

## Initialisation de l'agent

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

# On instancie le modèle (GPT-5 ou GPT-4o)
llm = ChatOpenAI(model="gpt-5", temperature=0)

# On crée l'agent : on lui donne le cerveau (LLM) et les mains (tools)
agent_executor = create_react_agent(llm, tools)

/tmp/ipython-input-3404887534.py:8: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_executor = create_react_agent(llm, tools)


## Testons l'agent en action

In [ ]:
query = "Combien font 4589 fois 12.5 ? Et combien de lettres compte le mot 'Anticonstitutionnellement' ?"

# L'agent va exécuter une boucle de réflexion
response = agent_executor.invoke({"messages": [("human", query)]})

# Affichons la réponse finale
print(response["messages"][-1].content)

- 4589 × 12,5 = 57 362,5
- Le mot "Anticonstitutionnellement" compte 25 lettres.
